In [4]:
import pandas as pd
import numpy as np
import cv2
import os 
import matplotlib.pyplot as plt
from tensorflow import keras
from keras import layers, models, optimizers
from sklearn.model_selection import train_test_split
import datetime
import urllib.request

In [8]:

# set your directories
train_path = '/kaggle/input/140k-real-and-fake-faces/real_vs_fake/real-vs-fake/train/'
val_path = '/kaggle/input/140k-real-and-fake-faces/real_vs_fake/real-vs-fake/valid/'
test_path = '/kaggle/input/140k-real-and-fake-faces/real_vs_fake/real-vs-fake/test/'

# define a function to load images
def load_images_from_directory(dir_path, labels='inferred', label_mode='binary', image_size=(256, 256), batch_size=32):
    dataset = keras.utils.image_dataset_from_directory(
        directory=dir_path,
        labels=labels,
        label_mode=label_mode,
        image_size=image_size,
        batch_size=batch_size,
        shuffle=True
    )
    return dataset

# load your datasets
train_dataset = load_images_from_directory(train_path)
val_dataset = load_images_from_directory(val_path)
test_dataset = load_images_from_directory(test_path)

Found 100000 files belonging to 2 classes.
Found 20000 files belonging to 2 classes.
Found 20000 files belonging to 2 classes.


In [9]:
def load_model_VGG16():
    model = keras.applications.VGG16(
        weights='imagenet',  # Load weights pre-trained on ImageNet.
        input_shape=(256, 256, 3),
        include_top=False
        )  # Do not include the ImageNet classifier at the top.
    return model

In [10]:
def set_nontrainable_layers(model):
    model.trainable = False
    return model

In [12]:
def add_last_layers(model):
    '''Take a pre-trained model, set its parameters as non-trainable, and add additional trainable layers on top'''
    rescale_layer = layers.Rescaling(1./255, input_shape=(256, 256, 3))
    base_model = set_nontrainable_layers(model)
    flatten_layer = layers.Flatten()
    dense_layer = layers.Dense(500, activation='relu')
    prediction_layer = layers.Dense(1, activation='sigmoid')
    
    
    model = models.Sequential([
        rescale_layer,
        base_model,
        flatten_layer,
        dense_layer,
        prediction_layer
    ])
    return model

In [15]:
def build_model():
    model = load_model_VGG16()
    model = add_last_layers(model)

    adam = optimizers.Adam(learning_rate=1e-4)

    model.compile(
        optimizer=adam,
        loss='binary_crossentropy',
        metrics=['accuracy', 'Precision', 'Recall']
    )
    return model

In [31]:
def save_model(model, name):
    model.save(f"./model/{name}" + '.h5')

def load_model(name):
    return models.load_model(f"./model/{name}" + '.h5')


In [26]:
tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()

# instantiate a distribution strategy
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.
INFO:tensorflow:Initializing the TPU system: local
INFO:tensorflow:Finished initializing TPU system.


INFO:tensorflow:Found TPU system:


INFO:tensorflow:Found TPU system:


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Cores: 8


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:6, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


In [27]:
def train_model(
        train_dataset,
        val_dataset,
        epochs=100,
        batch_size=32,
        patience=5
        ):
    es = keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=patience,
        restore_best_weights=True,
        )
    with tpu_strategy.scope():
        model = build_model()
    history = model.fit(
        train_dataset,
        epochs=epochs, 
        batch_size=batch_size, 
        validation_data=val_dataset,
        callbacks=[es]
        )
    accuracy = round(history.history['val_accuracy'][-1], 2)
    date = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    save_model(model, f"model_{accuracy}_{date}")
    return model, history

In [28]:

model, history = train_model(train_dataset, val_dataset, epochs=1000, batch_size=32, patience=5)

Epoch 1/1000


2023-06-06 13:28:25.202212: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Adam/Adam/AssignAddVariableOp.
2023-06-06 13:28:25.340216: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Adam/Adam/AssignAddVariableOp.


3124/3125 [============================>.] - ETA: 0s - loss: 0.3406 - accuracy: 0.8487 - precision: 0.8501 - recall: 0.8467

2023-06-06 13:29:55.327359: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Add/ReadVariableOp.
2023-06-06 13:29:55.447210: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Add/ReadVariableOp.


3125/3125 [==============================] - 116s 33ms/step - loss: 0.3405 - accuracy: 0.8487 - precision: 0.8501 - recall: 0.8467 - val_loss: 0.2615 - val_accuracy: 0.8887 - val_precision: 0.8611 - val_recall: 0.9268
Epoch 2/1000
3125/3125 [==============================] - 95s 30ms/step - loss: 0.2158 - accuracy: 0.9111 - precision: 0.9116 - recall: 0.9106 - val_loss: 0.2191 - val_accuracy: 0.9084 - val_precision: 0.9034 - val_recall: 0.9146
Epoch 3/1000
3125/3125 [==============================] - 94s 30ms/step - loss: 0.1626 - accuracy: 0.9350 - precision: 0.9349 - recall: 0.9351 - val_loss: 0.1963 - val_accuracy: 0.9201 - val_precision: 0.9198 - val_recall: 0.9206
Epoch 4/1000
3125/3125 [==============================] - 94s 30ms/step - loss: 0.1242 - accuracy: 0.9520 - precision: 0.9519 - recall: 0.9521 - val_loss: 0.1929 - val_accuracy: 0.9225 - val_precision: 0.9052 - val_recall: 0.9437
Epoch 5/1000
3125/3125 [==============================] - 95s 31ms/step - loss: 0.0936 - acc

In [33]:
accuracy = round(history.history['accuracy'][-1], 2)
date = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
save_model(model, f"model_{accuracy}_{date}")

In [34]:
res = model.evaluate(test_dataset)

625/625 [==============================] - 16s 23ms/step - loss: 0.1617 - accuracy: 0.9426 - precision: 0.9436 - recall: 0.9414


In [35]:
res

[0.16173754632472992,
 0.942550003528595,
 0.9435701966285706,
 0.9413999915122986]